In [ ]:
from utils.training_loop import set_seed, seed_worker, get_optimizer, train_one_epoch, evaluate_one_epoch
from utils.helper import save_imgs_df, get_std
from utils.transforms import get_transform
from utils.data import data_split, create_loader
from utils.model import get_model
from utils.training_loop import set_seed, seed_worker, get_optimizer, train_one_epoch, evaluate_one_epoch
from utils.checkpoint import save_best_model_weight, save_hparam, save_loss_log

import os
import time
import pandas as pd
import yaml
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder

In [ ]:
def main(cfg):
    # 시드 설정
    set_seed(cfg.hparam.train.seed)

    # mean, std
    meta_path = f'{cfg.data.root}/meta.csv'
    # save_imgs_df(cfg)

    df = pd.read_csv(meta_path)
    df.info()

    # 이미지별 픽셀 수 게산
    pixel_counts = df['width'] * df['height']

    # 전체 데이터셋 평균(mean) 계산
    print('# 전체 데이터셋 평균(mean) 계산')
    mean = (df['mean'] * pixel_counts).sum() / pixel_counts.sum()
    width = df['width'].mean()
    height = df['height'].mean()
    std = get_std(df, pixel_counts, mean)
    print(f'width:{int(width)} | height:{int(height)} mean: {mean} | std: {std}')


    # transform
    print('transform = get_transform(cfg.hparam.resize, mean, std): ')
    transform = get_transform(cfg.hparam.resize, mean, std)

    # DataSet(ImageFolder)
    print('# DataSet(ImageFolder)')
    dataset = ImageFolder(root=cfg.data.train_dir, transform=transform)

    # data split
    print('# data split: ')
    train_ds, val_ds, test_ds = data_split(dataset, [0.8, 0.1, 0.1])

    print('# dataloader')
    # dataloader
    print('train_loder')
    train_loader = create_loader(train_ds, cfg.hparam.train.batch_size, seed_worker,is_train=True, shuffle=True ,seed=cfg.hparam.train.seed)
    print('val_loader')
    val_loader = create_loader(val_ds, cfg.hparam.train.batch_size, seed_worker,is_train=True, shuffle=False ,seed=cfg.hparam.train.seed)
    print('test_loader')
    test_loader = create_loader(test_ds, cfg.hparam.train.batch_size, seed_worker,is_train=True, shuffle=False ,seed=cfg.hparam.train.seed)

    # model, device, criterion, optimizer
    print('device')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'device: {device}')

    print('model')
    model = get_model(cfg.hparam.models.resnet50, device)

    print('criterion')
    criterion = nn.CrossEntropyLoss()

    print('optimizer')
    optimizer = get_optimizer(cfg, model)

    # training loop
    print('best_loss')
    best_loss = float('inf')


    print('for epoch in range(1, cfg.hparam.train.epochs + 1):')
    for epoch in range(1, cfg.hparam.train.epochs + 1):
        print('train_loss, train_time')
        train_loss, train_time = train_one_epoch(model, train_loader, criterion,optimizer, device)


        print('# evaluate val')
        # evaluate val
        val_loss, val_accuracy, val_time = evaluate_one_epoch(model, val_loader, criterion, device)

        print('# evaluate test')
        # evaluate test
        test_loss, test_accuracy, test_time = evaluate_one_epoch(model, test_loader, criterion, device)
        print(f"Epoch {epoch}/{cfg.hparam.train.epochs}")
        print(f"Train loss: {train_loss:.4f} | {train_time}s")
        print(f"Valid loss: {val_loss:.4f} | Valid Accuracy: {val_accuracy:.2f} | {val_time}s")
        print(f"Test loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.2f} | {test_time}s")

        # save best model
        print('# save best model')
        if val_loss < best_loss:
            print('save_best_model_weight(')
            save_best_model_weight(model, val_loss, best_loss, cfg.project_root, cfg.exp_idx,filename="best_model.pth")

            print('save_hparam')
            save_hparam(cfg.hparam, cfg.exp_idx, cfg.project_root, filename='hparam.yaml')

            print('save_loss_log')
            save_loss_log(epoch, train_loss, val_loss, test_loss, cfg.exp_idx, cfg.project_root, filename="loss_log.csv")
            
            # yaml 파일 exp_idx + 1 늘리기
            print('# yaml 파일 exp_idx + 1 늘리기')
            cfg['exp+idx'] = cfg['exp_idx'] + 1
            print('open({cfg.project_root)')
            with open(f'{cfg.project_root}/configs/baseline.yaml', 'w') as f:
                yaml.dump(cfg, f, defalut_flow_style=False, sort_keys=False)



: 

In [ ]:
def run_train(cfg):
    main(cfg)